In [1]:
from Paper_Results import TVSSegmentation #as pr
import Segmantation_lib as sg
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dfFeat=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))
clstAlgo=['KMeans','specteral','affinity']
#kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
kc=[[9,6,6,4],[7,5,4,3],[10,7,9,6]]#Game of thrones
for s in range(1,2):
    #if s==1:
     #   ep=8
    #else:
     #   ep=14
    for e in range(9):
        print('season:',s,'episode:',e)
        #try:
        df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
        trSegem=TVSSegmentation(tvs="b",season=s,episode=e)
        framesFeatures, timeStamp,shot=trSegem.load_Data()
        manScene_boundry=trSegem.load_ManAnnotation()
        _,manShot_boundry=sg.getShots(shot)
        manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
        manScene_boundry=[i/1000 for i in manScene_boundry]
        avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
        randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
        #uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

        avgFeatures=np.array(avgFeatures)
        randSampleFeatures=np.array(randSampleFeatures)
        randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
        uniSmplFeatures=np.array(uniSmplFeatures)
        uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

        shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
        shot_TxtEmb=np.array(shot_TxtEmb)
        shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

        dist_AvgFeat=trSegem.getSimilarityMatrix(avgFeatures,'cosine')
        dist_RandFeat=trSegem.getSimilarityMatrix(randSampleFeatures,'cosine')
        dist_UniFeat=trSegem.getSimilarityMatrix(uniSmplFeatures,'cosine')
        dist_shot_TxtEmb=trSegem.getSimilarityMatrix(shot_TxtEmb,'cosine')

        manShot_boundry1=[i/1000 for i in manShot_boundry]
        shottimenp=np.array(manShot_boundry1[0:len(dist_AvgFeat)])
        shottimenp=shottimenp/np.linalg.norm(shottimenp)
        shottimenp=shottimenp.reshape(-1,1)
        dist_shottimenp=trSegem.getSimilarityMatrix(shottimenp,'cosine')
        dist_Visual=dist_AvgFeat
        dist_AvgFeat=dist_AvgFeat+dist_shot_TxtEmb
        dist_RandFeat=dist_RandFeat+dist_shot_TxtEmb
        dist_UniFeat=dist_UniFeat+dist_shot_TxtEmb

        #dist_AvgFeat=np.concatenate((dist_AvgFeat,dist_shot_TxtEmb),axis=1)
        #dist_RandFeat=np.concatenate((dist_RandFeat,dist_shot_TxtEmb),axis=1)
        #dist_UniFeat=np.concatenate((dist_UniFeat,dist_shot_TxtEmb),axis=1)
        #print(conctAvgFeat.shape, conctRandFeat.shape,conctUniFeat.shape)

        dist_AvgFeat_Time=dist_AvgFeat+dist_shottimenp
        dist_RandFeat_Time=dist_RandFeat+dist_shottimenp
        dist_UniFeat_Time=dist_UniFeat+dist_shottimenp

        #dist_AvgFeat_Time=np.concatenate((dist_AvgFeat,dist_shottimenp),axis=1)
        #dist_RandFeat_Time=np.concatenate((dist_RandFeat,dist_shottimenp),axis=1)
        #dist_UniFeat_Time=np.concatenate((dist_UniFeat,dist_shottimenp),axis=1)

        #dist_AvgFeat_Time=np.column_stack((dist_AvgFeat,dist_shottimenp))
        #dist_RandFeat_Time=np.column_stack((dist_RandFeat,dist_shottimenp))
        #dist_UniFeat_Time=np.column_stack((dist_UniFeat,dist_shottimenp))
        #conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

        #dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
        #dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
        #dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')


        for i, clAl in enumerate(clstAlgo):
            visClusters=trSegem.getClusters(40,clAl,dist_Visual)
            avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
            randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
            uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
            avgFeatClusters=avgFeatClusters.tolist()
            randFeatClusters=randFeatClusters.tolist()
            uniFeatClusters=uniFeatClusters.tolist()
            avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
            randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
            uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
            avgFeatClusters_Time=avgFeatClusters_Time.tolist()
            randFeatClusters_Time=randFeatClusters_Time.tolist()
            uniFeatClusters_Time=uniFeatClusters_Time.tolist()
            visClusters=visClusters.tolist()

            sepVisual=sg.sequenceSegmentation(visClusters,kc[i][0],kc[i][1])
            sepPos=sg.sequenceSegmentation(avgFeatClusters,kc[i][0],kc[i][1])
            sepPos1=sg.sequenceSegmentation(randFeatClusters,kc[i][0],kc[i][1])
            sepPos2=sg.sequenceSegmentation(uniFeatClusters,kc[i][0],kc[i][1])
            sceneTime=[]
            sceneTime1=[]
            sceneTime2=[]
            sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,kc[i][2],kc[i][3])
            sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,kc[i][2],kc[i][3])
            sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,kc[i][2],kc[i][3])
            sceneTime_Time=[]
            sceneTime1_Time=[]
            sceneTime2_Time=[]
            sceneTimeVis=[]
            for i in sepVisual:
                sceneTimeVis.append(manShot_boundry[i])
            for i in sepPos:
                sceneTime.append(manShot_boundry[i])
            for i in sepPos1:
                sceneTime1.append(manShot_boundry[i])
            for i in sepPos2:
                sceneTime2.append(manShot_boundry[i])
            for i in sepPos2_Time:
                sceneTime2.append(manShot_boundry[i])
            for i in sepPos_Time:
                sceneTime.append(manShot_boundry[i])
            for i in sepPos1_Time:
                sceneTime1.append(manShot_boundry[i])
            print('Auto',clAl,':',len(sepVisual),len(sepPos),len(sepPos1),len(sepPos_Time),len(sepPos1_Time),'the Manual:',len(manScene_boundry))
            tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
            len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)
            tAutoVis,tAuto_binVis,tAuto_bin_old_Vis,_=trSegem.getTruthValues_Auto(sepVisual,manShot_boundry)
            tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
            tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
            tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
            tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
            tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
            tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)

            truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
            truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,Auto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2]
            truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]

            results=trSegem.saveEvaluations(tMan,tAutoVis,rType='macro')
            row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfFeat = dfFeat.append(row)
            results=trSegem.saveEvaluations(tMan_bin,tAuto_binVis,rType='binary')
            row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfFeat = dfFeat.append(row)

            for i in range(len(truth_valueList)):
                if i%3==0:
                    results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
                else:
                    results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
                row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                df = df.append(row)
                df_tmp=df_tmp.append(row)

            row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            df_tmp=df_tmp.drop([clAl])

            for i in range(len(truth_valueList_Time)):
                if i%3==0:
                    results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
                else:
                    results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
                row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                df1 = df1.append(row)
                df_tmp=df_tmp.append(row)

            row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
            dfBest=dfBest.append(row1)
            df_tmp=df_tmp.drop([clAl])
        #except:
        #    continue

season: 3 episode: 1
You have chosen :
Tv-Series: Breaking bad
Season:  3
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/BB/English/BreakingBad.Season03.Episode01.en.srt


In [5]:
dfFeat.to_csv('ResultsPaper/BB/Results_segmentaion_JustVisual_Samp_test_Late.csv')
df.to_csv('ResultsPaper/BB/Results_segmentaion_TextEmbe_Augemnted_Samp_test_Late.csv')
df1.to_csv('ResultsPaper/BB/Results_segmentaion_TimeAugmented_Samp_test_Late.csv')
dfBest.to_csv('ResultsPaper/BB/Results_segmentaion_Features_Samp_test_Late.csv')

In [4]:
dfFeat=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
df1=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
dfBest=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision','season','episode'))
clstAlgo=['KMeans','specteral','affinity']
#kc=[[7,5,6,4],[6,4,3,3],[9,7,7,5]]
kc=[[9,6,6,4],[7,5,4,3],[10,7,9,6]]#Game of thrones
for s in range(4,6):
    #if s==1:
        #ep=8
    #else:
      #  ep=14
    for e in range(1,11):
        print('season:',s,'episode:',e)
        try:
            df_tmp=pd.DataFrame(columns=('winDiff','pk','coverage','purity','recall','precision'))
            trSegem=TVSSegmentation(tvs="g",season=s,episode=e)
            framesFeatures, timeStamp,shot=trSegem.load_Data()
            manScene_boundry=trSegem.load_ManAnnotation()
            _,manShot_boundry=sg.getShots(shot)
            manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
            manScene_boundry=[i/1000 for i in manScene_boundry]
            avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
            randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
            #uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

            avgFeatures=np.array(avgFeatures)
            randSampleFeatures=np.array(randSampleFeatures)
            randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
            uniSmplFeatures=np.array(uniSmplFeatures)
            uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

            shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
            shot_TxtEmb=np.array(shot_TxtEmb)
            shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

            dist_AvgFeat=trSegem.getSimilarityMatrix(avgFeatures,'cosine')
            dist_RandFeat=trSegem.getSimilarityMatrix(randSampleFeatures,'cosine')
            dist_UniFeat=trSegem.getSimilarityMatrix(uniSmplFeatures,'cosine')
            dist_shot_TxtEmb=trSegem.getSimilarityMatrix(shot_TxtEmb,'cosine')

            manShot_boundry1=[i/1000 for i in manShot_boundry]
            shottimenp=np.array(manShot_boundry1[0:len(dist_AvgFeat)])
            shottimenp=shottimenp/np.linalg.norm(shottimenp)
            shottimenp=shottimenp.reshape(-1,1)
            dist_shottimenp=trSegem.getSimilarityMatrix(shottimenp,'cosine')
            dist_Visual=dist_AvgFeat
            dist_AvgFeat=dist_AvgFeat+dist_shot_TxtEmb
            dist_RandFeat=dist_RandFeat+dist_shot_TxtEmb
            dist_UniFeat=dist_UniFeat+dist_shot_TxtEmb

            #dist_AvgFeat=np.concatenate((dist_AvgFeat,dist_shot_TxtEmb),axis=1)
            #dist_RandFeat=np.concatenate((dist_RandFeat,dist_shot_TxtEmb),axis=1)
            #dist_UniFeat=np.concatenate((dist_UniFeat,dist_shot_TxtEmb),axis=1)
            #print(conctAvgFeat.shape, conctRandFeat.shape,conctUniFeat.shape)

            dist_AvgFeat_Time=dist_AvgFeat+dist_shottimenp
            dist_RandFeat_Time=dist_RandFeat+dist_shottimenp
            dist_UniFeat_Time=dist_UniFeat+dist_shottimenp

            #dist_AvgFeat_Time=np.concatenate((dist_AvgFeat,dist_shottimenp),axis=1)
            #dist_RandFeat_Time=np.concatenate((dist_RandFeat,dist_shottimenp),axis=1)
            #dist_UniFeat_Time=np.concatenate((dist_UniFeat,dist_shottimenp),axis=1)

            #dist_AvgFeat_Time=np.column_stack((dist_AvgFeat,dist_shottimenp))
            #dist_RandFeat_Time=np.column_stack((dist_RandFeat,dist_shottimenp))
            #dist_UniFeat_Time=np.column_stack((dist_UniFeat,dist_shottimenp))
            #conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

            #dist_AvgFeat_Time=trSegem.getSimilarityMatrix(conctAvgFeat_Time,'cosine')
            #dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
            #dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')


            for i, clAl in enumerate(clstAlgo):
                visClusters=trSegem.getClusters(40,clAl,dist_Visual)
                avgFeatClusters=trSegem.getClusters(40,clAl,dist_AvgFeat)
                randFeatClusters=trSegem.getClusters(40,clAl,dist_RandFeat)
                uniFeatClusters=trSegem.getClusters(40,clAl,dist_UniFeat)
                avgFeatClusters=avgFeatClusters.tolist()
                randFeatClusters=randFeatClusters.tolist()
                uniFeatClusters=uniFeatClusters.tolist()
                avgFeatClusters_Time=trSegem.getClusters(40,clAl,dist_AvgFeat_Time)
                randFeatClusters_Time=trSegem.getClusters(40,clAl,dist_RandFeat_Time)
                uniFeatClusters_Time=trSegem.getClusters(40,clAl,dist_UniFeat_Time)
                avgFeatClusters_Time=avgFeatClusters_Time.tolist()
                randFeatClusters_Time=randFeatClusters_Time.tolist()
                uniFeatClusters_Time=uniFeatClusters_Time.tolist()
                visClusters=visClusters.tolist()

                sepVisual=sg.sequenceSegmentation(visClusters,kc[i][0],kc[i][1])
                sepPos=sg.sequenceSegmentation(avgFeatClusters,kc[i][0],kc[i][1])
                sepPos1=sg.sequenceSegmentation(randFeatClusters,kc[i][0],kc[i][1])
                sepPos2=sg.sequenceSegmentation(uniFeatClusters,kc[i][0],kc[i][1])
                sceneTime=[]
                sceneTime1=[]
                sceneTime2=[]
                sepPos_Time=sg.sequenceSegmentation(avgFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos1_Time=sg.sequenceSegmentation(randFeatClusters_Time,kc[i][2],kc[i][3])
                sepPos2_Time=sg.sequenceSegmentation(uniFeatClusters_Time,kc[i][2],kc[i][3])
                sceneTime_Time=[]
                sceneTime1_Time=[]
                sceneTime2_Time=[]
                sceneTimeVis=[]
                for i in sepVisual:
                    sceneTimeVis.append(manShot_boundry[i])
                for i in sepPos:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1:
                    sceneTime1.append(manShot_boundry[i])
                for i in sepPos2:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos2_Time:
                    sceneTime2.append(manShot_boundry[i])
                for i in sepPos_Time:
                    sceneTime.append(manShot_boundry[i])
                for i in sepPos1_Time:
                    sceneTime1.append(manShot_boundry[i])
                print('Auto',clAl,':',len(sepVisual),len(sepPos),len(sepPos1),len(sepPos_Time),len(sepPos1_Time),'the Manual:',len(manScene_boundry))
                tMan,tMan_bin,tMan_bin_old,sceneTime_man_bin=trSegem.getTruthValue_Man(manShot_boundry,manScene_boundry)
                len(tMan),len(tMan_bin),len(tMan_bin_old),len(sceneTime_man_bin)
                tAutoVis,tAuto_binVis,tAuto_bin_old_Vis,_=trSegem.getTruthValues_Auto(sepVisual,manShot_boundry)
                tAuto,tAuto_bin,tAuto_bin_old,sceneTime_Auto_bin=trSegem.getTruthValues_Auto(sepPos,manShot_boundry)
                tAuto1,tAuto_bin1,tAuto_bin_old1,sceneTime_Auto_bin1=trSegem.getTruthValues_Auto(sepPos1,manShot_boundry)
                tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,sceneTime_Auto_bin_Time=trSegem.getTruthValues_Auto(sepPos_Time,manShot_boundry)
                tAuto2,tAuto_bin2,tAuto_bin_old2,sceneTime_Auto_bin2=trSegem.getTruthValues_Auto(sepPos2,manShot_boundry)
                tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2,sceneTime_Auto_bin_Time2=trSegem.getTruthValues_Auto(sepPos2_Time,manShot_boundry)
                tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,sceneTime_Auto_bin1_Time=trSegem.getTruthValues_Auto(sepPos1_Time,manShot_boundry)

                truth_valueList=[tAuto,tAuto_bin,tAuto_bin_old,tAuto1,tAuto_bin1,tAuto_bin_old1,tAuto2,tAuto_bin2,tAuto_bin_old2]
                truth_valueList_Time=[tAuto_Time,tAuto_bin_Time,tAuto_bin_old_Time,tAuto1_Time,tAuto_bin1_Time,tAuto_bin_old1_Time,tAuto_Time2,tAuto_bin_Time2,tAuto_bin_old_Time2]
                truth_valueListMan=[tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old,tMan,tMan_bin,tMan_bin_old]

                results=trSegem.saveEvaluations(tMan,tAutoVis,rType='macro')
                row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfFeat = dfFeat.append(row)
                results=trSegem.saveEvaluations(tMan_bin,tAuto_binVis,rType='binary')
                row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfFeat = dfFeat.append(row)

                for i in range(len(truth_valueList)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df = df.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])

                for i in range(len(truth_valueList_Time)):
                    if i%3==0:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='macro')
                    else:
                        results=trSegem.saveEvaluations(truth_valueListMan[i],truth_valueList_Time[i],rType='binary')
                    row=pd.Series({'winDiff':results[0], 'pk':results[1],'coverage':results[2],'purity':results[3],'recall':results[4],'precision':results[5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                    df1 = df1.append(row)
                    df_tmp=df_tmp.append(row)

                row1=pd.Series({'winDiff':df_tmp.iloc[1][0], 'pk':df_tmp.iloc[1][1],'coverage':df_tmp.iloc[0][2],'purity':df_tmp.iloc[0][3],'recall':df_tmp.iloc[1][4],'precision':df_tmp.iloc[1][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[4][0], 'pk':df_tmp.iloc[4][1],'coverage':df_tmp.iloc[3][2],'purity':df_tmp.iloc[3][3],'recall':df_tmp.iloc[4][4],'precision':df_tmp.iloc[4][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                row1=pd.Series({'winDiff':df_tmp.iloc[7][0], 'pk':df_tmp.iloc[7][1],'coverage':df_tmp.iloc[6][2],'purity':df_tmp.iloc[6][3],'recall':df_tmp.iloc[7][4],'precision':df_tmp.iloc[7][5],'season':str(trSegem.season),'episode':str(trSegem.episode)},name=clAl)
                dfBest=dfBest.append(row1)
                df_tmp=df_tmp.drop([clAl])
        except:
            continue

season: 4 episode: 1
You have chosen :
Tv-Series: Game of Thrones
Season:  4
Episode: 1
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season04.Episode01.en.srt
Auto KMeans : 31 43 36 79 84 the Manual: 22
Auto specteral : 27 38 24 97 101 the Manual: 22
Auto affinity : 32 29 36 35 47 the Manual: 22
season: 4 episode: 2
You have chosen :
Tv-Series: Game of Thrones
Season:  4
Episode: 2
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season04.Episode02.en.srt
Auto KMeans : 28 33 32 70 68 the Manual: 16
Auto specteral : 25 29 33 90 108 the Manual: 16
Auto affinity : 31 28 38 36 50 the Manual: 16
season: 4 episode: 3
You have chosen :
Tv-Series: Game of Thrones
Season:  4
Episode: 3
Subtile File :  /people/berhe/Bureau/TLP_thesis/subtitles/GoT/English/GameOfThrones.Season04.Episode03.en.srt
Auto KMeans : 28 35 29 79 72 the Manual: 19
Auto specteral : 26 33 49 79 87 the Manual: 19
Auto affinity : 31 30 36 41 49 the Manu

In [5]:
dfFeat.to_csv('ResultsPaper/GoT/Results_segmentaion_JustVisual_Samp_test_Late.csv')
df.to_csv('ResultsPaper/GoT/Results_segmentaion_TextEmbe_Augemnted_Samp_test_Late.csv')
df1.to_csv('ResultsPaper/GoT/Results_segmentaion_TimeAugmented_Samp_test_Late.csv')
dfBest.to_csv('ResultsPaper/GoT/Results_segmentaion_Features_Samp_test.csv_Late.csv')

In [9]:
dfBest

,winDiff,pk,coverage,purity,recall,precision,season,episode
KMeans,0.135,0.071,0.682,0.895,0.957,0.969,3,1
KMeans,0.109,0.062,0.790,0.927,0.963,0.973,3,1
KMeans,0.105,0.058,0.717,0.906,0.965,0.975,3,1
KMeans,0.150,0.086,0.633,0.925,0.940,0.970,3,1
KMeans,0.139,0.077,0.566,0.933,0.946,0.974,3,1
KMeans,1.805,0.914,0.597,0.929,0.060,0.912,3,1
specteral,0.094,0.052,0.788,0.854,0.973,0.973,3,1
specteral,0.113,0.060,0.702,0.792,0.969,0.969,3,1
specteral,0.109,0.056,0.738,0.807,0.971,0.971,3,1
specteral,0.176,0.101,0.504,0.957,0.919,0.975,3,1


In [ ]:

manClstr_ref,sceneMakar=trSegem.frameGroundClusters(manShot_boundry)
manScene_boundry=[i/1000 for i in manScene_boundry]
avgFeatures=trSegem.getAvgFeatures(framesFeatures,timeStamp,manShot_boundry)
randSampleFeatures,uniSmplFeatures=trSegem.getRandomFeat(framesFeatures,timeStamp,manShot_boundry)
#uniSmplFeatures=trSegem.getUniformSamples(framesFeatures,timeStamp,manShot_boundry)

avgFeatures=np.array(avgFeatures)
randSampleFeatures=np.array(randSampleFeatures)
randSampleFeatures=randSampleFeatures.reshape((randSampleFeatures.shape[0],randSampleFeatures.shape[1]*randSampleFeatures.shape[2]))
uniSmplFeatures=np.array(uniSmplFeatures)
uniSmplFeatures=uniSmplFeatures.reshape((uniSmplFeatures.shape[0],uniSmplFeatures.shape[1]*uniSmplFeatures.shape[2]))

shot_TxtEmb=trSegem.getShotTextFeat(manShot_boundry)
shot_TxtEmb=np.array(shot_TxtEmb)
shot_TxtEmb.shape,avgFeatures.shape,randSampleFeatures.shape

dist_AvgFeat=trSegem.getSimilarityMatrix(avgFeatures,'cosine')
dist_RandFeat=trSegem.getSimilarityMatrix(randSampleFeatures,'cosine')
dist_UniFeat=trSegem.getSimilarityMatrix(uniSmplFeatures,'cosine')
dist_shot_TxtEmb=trSegem.getSimilarityMatrix(shot_TxtEmb,'cosine')

manShot_boundry1=[i/1000 for i in manShot_boundry]
shottimenp=np.array(manShot_boundry1[0:len(dist_AvgFeat)])
shottimenp=shottimenp/np.linalg.norm(shottimenp)
shottimenp=shottimenp.reshape(-1,1)
dist_shottimenp=trSegem.getSimilarityMatrix(shottimenp,'cosine')


dist_AvgFeat=np.concatenate((dist_AvgFeat,dist_shot_TxtEmb),axis=1)
dist_RandFeat=np.concatenate((dist_RandFeat,dist_shot_TxtEmb),axis=1)
dist_UniFeat=np.concatenate((dist_UniFeat,dist_shot_TxtEmb),axis=1)
#print(conctAvgFeat.shape, conctRandFeat.shape,conctUniFeat.shape)

dist_AvgFeat_Time=np.concatenate((dist_AvgFeat,dist_shottimenp),axis=1)
dist_RandFeat_Time=np.concatenate((dist_RandFeat,dist_shottimenp),axis=1)
dist_UniFeat_Time=np.concatenate((dist_UniFeat,dist_shottimenp),axis=1)

#dist_AvgFeat_Time=np.column_stack((dist_AvgFeat,dist_shottimenp))
#conctRandFeat_Time=np.column_stack((dist_RandFeat,dist_shottimenp))
#conctUniFeat_Time=np.column_stack((dist_UniFeat,dist_shottimenp))
#conctAvgFeat_Time.shape, conctRandFeat_Time.shape,shottimenp.shape

#dist_AvgFeat_Time=trSegem.getSimilarityMatrix(dist_AvgFeat_Time,'cosine')
#dist_RandFeat_Time=trSegem.getSimilarityMatrix(conctRandFeat_Time,'cosine') 
#dist_UniFeat_Time=trSegem.getSimilarityMatrix(conctUniFeat_Time,'cosine')

In [6]:
shottimenp1=shottimenp.reshape(-1,1)

In [8]:
dist_shottimenp=trSegem.getSimilarityMatrix(shottimenp1,'cosine')

In [17]:
conctUniFeat_Time.shape,dist_UniFeat.shape,dist_shot_TxtEmb.shape

((980, 4900), (980, 3920), (980, 980))

In [16]:
dist_AvgFeat.shape,dist_AvgFeat_Time.shape,dist_RandFeat.shape,dist_RandFeat_Time.shape,dist_UniFeat.shape,dist_UniFeat_Time.shape

((980, 1960), (980, 2940), (980, 1960), (980, 2940), (980, 1960), (980, 2940))

In [ ]:
df_tmp